In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv() 

True

## Cycle Graph Creation

In [2]:
from langchain.prompts import PromptTemplate

cycle_graph_generation_prompt = PromptTemplate.from_template(
    '''
    Create a natural dialogue graph that represents any business conversation with logical cycles.

    Here's an example of a clothing store conversation flow:

    ```mermaid
    graph TD
        1[Welcome] -->|"Looking for jeans"| 2[Ask Preferences]
        2 -->|"Slim fit, blue"| 3[Offer Options]
        3 -->|"Like the first one"| 4[Size Check]
        4 -->|"Size 32"| 5[Location]
        5 -->|"Try in store"| 6[Complete]
        6 -->|"Look for something else"| 2
    ```

    Example JSON structure:
    {{
        "edges": [
            {{ "source": 1, "target": 2, "utterances": ["I'm looking for jeans"] }},
            {{ "source": 2, "target": 3, "utterances": ["I'd like slim fit in blue"] }},
            {{ "source": 3, "target": 4, "utterances": ["I like the first option"] }},
            {{ "source": 4, "target": 5, "utterances": ["Size 32 please"] }},
            {{ "source": 5, "target": 6, "utterances": ["I'll try them in store"] }},
            {{ "source": 6, "target": 2, "utterances": ["I'd like to look for another pair"] }}
        ],
        "nodes": [
            {{ "id": 1, "label": "welcome", "is_start": true,
              "utterances": ["Welcome to Style Store! How can I help you today?"] }},
            {{ "id": 2, "label": "ask_preferences", "is_start": false,
              "utterances": ["What style and color are you interested in?"] }},
            {{ "id": 3, "label": "offer_options", "is_start": false,
              "utterances": ["I have these slim fit blue jeans available. Would you like to see them?"] }},
            {{ "id": 4, "label": "check_size", "is_start": false,
              "utterances": ["What size would you like?"] }},
            {{ "id": 5, "label": "location_preference", "is_start": false,
              "utterances": ["Would you like to try them in store or have them delivered?"] }},
            {{ "id": 6, "label": "complete", "is_start": false,
              "utterances": ["Perfect! The jeans are ready for you. Would you like to look for anything else?"] }}
        ]
    }}

    Key points:
    1. Keep responses natural and contextual
    2. Each state should serve a clear purpose
    3. Return cycles should make conversational sense

    Create a dialogue graph for topic: {topic}
    Return only valid JSON matching the format above.
    '''
)

In [ ]:
from chatsky_llm_autoconfig.algorithms.dialogue_augmentation import DialogAugmentation
from chatsky_llm_autoconfig.algorithms.dialogue_generation import DialogueSampler
from chatsky_llm_autoconfig.algorithms.topic_graph_generation import CycleGraphGenerator
from chatsky_llm_autoconfig.metrics.llm_metrics import are_triplets_valid
from typing import List, Dict, Any
import os
import json
from datetime import datetime
from pathlib import Path

def generate_dialogues(
    topics: List[str], 
) -> List[Dict[str, Any]]:

    # Initialize components
    graph_generator = CycleGraphGenerator(prompt=cycle_graph_generation_prompt)
    sampler = DialogueSampler()
    
    model = ChatOpenAI(
        model='gpt-4o', 
        api_key=os.getenv("OPENAI_API_KEY"), 
        base_url=os.getenv("OPENAI_BASE_URL"), 
        temperature=0
    )
    
    results = []
    
    for topic in topics:
        # Generate graph and validate
        graph = graph_generator.invoke(topic=topic)
        is_valid = are_triplets_valid(graph, model, topic)
        
        if is_valid["value"]:
            print(f"Graph: {graph}")
            sampled_dialogue = sampler.invoke(graph, 1, -1)[0]
            
            print(f"Sampled dialogue: {sampled_dialogue}")
            
            # Initialize dialogues array with the sampled dialogue
            dialogues = [sampled_dialogue.model_dump()]
        
            result_entry = {
                "graph": graph.graph_dict,
                "topic": topic,
                "dialogues": dialogues
            }
            
            results.append(result_entry)
            
    return results

topics = [
   "restaurant table reservation",
   "tech support troubleshooting",
   "hotel room booking", 
   "car service appointment",
   "flight ticket booking",
   "gym membership registration",
   "real estate property viewing",
   "insurance policy purchase",
   "beauty salon appointment",
   "bank account opening"
]

dialogue_collection = generate_dialogues(topics)


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [ ]:
len(dialogue_collection)

6